Grid post-processing of Met.no data using SHyFT API
=========

### This notebook gives an example of Met.no data post-processing to correct temperature forecasts based on comparison to observations. The following steps are described:
1. **Loading required python modules and setting path to SHyFT installation**
2. **Generate synthetic data for temperature observation time-series**
3. **Transform observations from set to grid (IDW)**
4. **Create forecasts on the same grid**
5. **Transform forecasts from grid to set (Kriging)**
6. **Calculate the bias time-series using Kalman filter on the observation set**
7. **Transform bias from set to grid (IDW) and apply bias to forecasts**
8. **Transform corrected forecasts from grid to set (Kriging)**
9. **Plot the results**

### 1. Loading required python modules and setting path to SHyFT installation

In [ ]:
# first you should import the third-party python modules which you'll use later on
# the first line enables that figures are shown inline, directly in the notebook
%pylab inline
import os
from os import path
import sys
from matplotlib import pyplot as plt

In [ ]:
# set the path for your shyft build
# this should point to the directory that is created
# when you clone shyft, assuming you have built shyft
# there and not installed it to your system python
shyft_path = os.path.abspath("../../../shyft")
sys.path.insert(0, shyft_path)

# you could achieve the same by setting a PYTHONPATH

In [ ]:
# once the shyft_path is set correctly, you should be able to import shyft modules
import shyft
from shyft import shyftdata_dir

# if you have problems here, it may be related to having your LD_LIBRARY_PATH
# pointing to the appropriate libboost_python libraries (.so files)
from shyft.repository.default_state_repository import DefaultStateRepository
from shyft.orchestration.configuration import yaml_configs
from shyft.orchestration.simulators.config_simulator import ConfigSimulator
from shyft import api

In [ ]:
# now you can access the api of shyft with tab completion and help, try this:

#help(api.GeoPoint) # remove the hashtag and run the cell to print the documentation of the api.GeoPoint class
#api. # remove the hashtag, set the pointer behind the dot and use 
      # tab completion to see the available attributes of the shyft api

### 2. Generate synthetic data for temperature observation time-series

In [ ]:
# Create time-axis
t0 = api.Calendar().time(2016, 1, 1)
ta = api.Timeaxis(t0, api.deltahours(1), 240)
ta2 = api.Timeaxis2(t0, api.deltahours(1), 240)

# Create a TemperatureSourceVector to hold the set of observation time-series
obs_set = api.TemperatureSourceVector()

# Create time-series having a constant temperature of 15 and add them to the observation set
ts = api.Timeseries(ta2, fill_value=15.0)
obs_set.append(api.TemperatureSource(api.GeoPoint( 100, 100, 1000), ts))
obs_set.append(api.TemperatureSource(api.GeoPoint(5100, 100, 1150), ts))
obs_set.append(api.TemperatureSource(api.GeoPoint( 100, 5100, 850), ts))
obs_set.append(api.TemperatureSource(api.GeoPoint(5100, 5100, 900), ts))

### 3. Transform observations from set to grid (IDW)

In [ ]:
# Create a target grid specification of 1 x 1 km
gpv_1 = api.GeoPointVector()
for x in range(10):
    for y in range(10):
        gpv_1.append(api.GeoPoint(x*1000, y*1000, (x+y)*50))
        
# Generate the observation grid by IDW transform of temperature model
obs_grid = api.idw_temperature(obs_set, gpv_1, ta, api.IDWTemperatureParameter())

### 4. Create forecasts on the same grid

In [ ]:
# Create a forecast grid by copying the observation grid
fc_grid = obs_grid

# Add a time-series of constant value to every forecast time-series
const_ts = api.Timeseries(ta2, fill_value=2.0)
for fc in fc_grid:
    fc.ts += const_ts

### 5. Transform forecasts from grid to set (Kriging)

In [ ]:
# Create the target set specification at same GeoPoint as observation set
gpv_2 = api.GeoPointVector()
gpv_2.append(api.GeoPoint( 100, 100, 1000))
gpv_2.append(api.GeoPoint(5100, 100, 1150))
gpv_2.append(api.GeoPoint( 100, 5100, 850))
gpv_2.append(api.GeoPoint(5100, 5100, 900))
        
# Create the forecast set by Krieging transform of temperature model
fc_set = api.bayesian_kriging_temperature(fc_grid, gpv_2, ta, api.BTKParameter())

### 6. Calculate the bias time-series using Kalman filter on the observation set

In [ ]:
# Create a TemperatureSourceVector to hold the set of bias time-series
bias_set = api.TemperatureSourceVector()

# Create the Kalman filter having 8 samples spaced every 3 hours to represent a daily periodic pattern
kf = api.KalmanFilter()
kbp = api.KalmanBiasPredictor(kf)
kta = api.Timeaxis2(t0, api.deltahours(3), 8)

# Calculate the coefficients of Kalman filter and 
# Create bias time-series based on the daily periodic pattern
for obs in obs_set:
    kbp.update_with_forecast(fc_set, obs.ts, kta)
    pattern = api.KalmanState.get_x(kbp.state) * np.array(-1.0) # By convention, inverse sign of pattern values
    bias_ts = api.create_periodic_pattern_ts(pattern, api.deltahours(3), ta.time(0), ta2)
    bias_set.append(api.TemperatureSource(obs.mid_point(), bias_ts))

### 7. Transform bias from set to grid (IDW) and apply to forecast grid

In [ ]:
# Generate the bias grid by IDW transform of temperature model
bias_grid = api.idw_temperature(bias_set, gpv_1, ta, api.IDWTemperatureParameter())

# Correct forecasts by applying bias time-series on the grid
for i in range(len(fc_grid)):
    fc_grid[i].ts += bias_grid[i].ts # By convention, add bias time-series

In [ ]:
# Check the last value of the time-series. It should be around 15
print(fc_grid[0].ts.value(239))

### 8. Transform corrected forecasts from grid to set (Kriging)

In [ ]:
# Generate the corrected forecast set by Krieging transform of temperature model
fc_set = api.bayesian_kriging_temperature(fc_grid, gpv_2, ta, api.BTKParameter())

### 9. Plot the results

In [ ]:
# Make a time-series plot of temperature sets
fig, ax = plt.subplots(figsize=(20, 10))
for i in range(len(bias_set)):
    timestamps = [datetime.datetime.utcfromtimestamp(p.start) for p in obs_set[i].ts.time_axis]
    ax.plot(timestamps, obs_set[i].ts.values, label = str(i+1) + ' Observation')
    ax.plot(timestamps, fc_set[i].ts.values, label = str(i+1) + ' Forecast')
    ax.plot(timestamps, bias_set[i].ts.values, label = str(i+1) + ' Bias')
fig.autofmt_xdate()
ax.legend(title='Temperature')
ax.set_ylabel('Temp ($^\circ$C)')

In [ ]:
# Make a scatter plot of grid temperature forecasts at ts.value(0)
x = [fc.mid_point().x for fc in fc_grid]
y = [fc.mid_point().y for fc in fc_grid]
fig, ax = plt.subplots(figsize=(10, 5))
temps = np.array([fc.ts.value(0) for fc in fc_grid])
plot = ax.scatter(x, y, c=temps, marker='s', vmin=0, vmax=20, s=500, lw=0)
plt.colorbar(plot).set_label('Temp ($^\circ$C)')